In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob
import kagglehub

In [ ]:
# Download dataset
path = kagglehub.dataset_download("tr1gg3rtrash/time-series-top-100-crypto-currency-dataset")

In [ ]:
# Output folder
output_folder = r"D:\crypto_out"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Function to process and plot each file
def process_crypto_file(file_path):
    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.sort_values('timestamp')

        # Calculate daily returns without fill_method
        df['daily_return'] = df['close'].pct_change(fill_method=None)

        # Rolling volatility (7-day)
        df['rolling_volatility'] = df['daily_return'].rolling(window=7).std()

        # Volatility thresholds
        high_threshold = df['rolling_volatility'].quantile(0.75)
        low_threshold = df['rolling_volatility'].quantile(0.25)

        # Label volatility levels
        df['volatility_level'] = 'Normal'
        df.loc[df['rolling_volatility'] >= high_threshold, 'volatility_level'] = 'High Volatility'
        df.loc[df['rolling_volatility'] <= low_threshold, 'volatility_level'] = 'Low Volatility'

        # Plotting
        plt.figure(figsize=(16, 8))
        sns.lineplot(data=df, x='timestamp', y='close', hue='volatility_level',
                     palette={'High Volatility': 'red', 'Low Volatility': 'green', 'Normal': 'blue'})
        plt.title(f"{os.path.basename(file_path).replace('.csv', '')} Price with Volatility")
        plt.xlabel('Date')
        plt.ylabel('Closing Price')
        plt.legend(title='Volatility Level')
        plt.grid(True)
        plt.tight_layout()

        # Save plot
        filename = os.path.basename(file_path).replace(".csv", "_volatility.png")
        plt.savefig(os.path.join(output_folder, filename))
        plt.close()
        print(f"✅ Saved: {filename}")

    except Exception as e:
        print(f"❌ Error in {file_path}: {e}")

In [ ]:
# Process all CSVs
all_csv_files = glob(os.path.join(path, "*.csv"))
for file in all_csv_files:
    process_crypto_file(file)